In [ ]:
import time
import subprocess
from pathlib import Path
import pandas as pd
from ultralytics import YOLO

# ---------------- CONFIG ----------------
IDD_ROOT = Path("../../data/idd_temporal_test_1")
MODEL_PATH = "../../runs/three_country_training/road_defect_binary/yolov8s/weights/best.pt"

OUTPUT_ROOT = Path("../../runs/idd_temporal_inference/yolov8s")

IMG_SIZE = 640
CONF = 0.25
IOU = 0.7
VID_STRIDE = 1
# ---------------------------------------

model = YOLO(MODEL_PATH)

records = []

# Iterate over sequence IDs (e.g., 00082, 00083, ...)
for seq_id_dir in sorted(IDD_ROOT.iterdir()):
    if not seq_id_dir.is_dir():
        continue

    # Each subfolder inside is a temporal clip
    for clip_dir in sorted(seq_id_dir.iterdir()):
        if not clip_dir.is_dir():
            continue

        clip_name = f"{seq_id_dir.name}_{clip_dir.name}"
        print(f"\n▶ Processing sequence: {clip_name}")

        frame_count = 0
        start = time.time()

        results = model.predict(
            source=str(clip_dir),     # 👈 directory of frames
            imgsz=IMG_SIZE,
            conf=CONF,
            iou=IOU,
            vid_stride=VID_STRIDE,
            stream=True,
            save=True,
            project=str(OUTPUT_ROOT),
            name=clip_name
        )

        for _ in results:
            frame_count += 1

        end = time.time()
        elapsed = end - start
        fps = frame_count / elapsed if elapsed > 0 else 0

        print(f"   Frames: {frame_count} | Time: {elapsed:.2f}s | FPS: {fps:.2f}")

        # ---------------- AVI → MP4 ----------------
        run_dir = OUTPUT_ROOT / clip_name
        avi_files = list(run_dir.glob("*.avi"))

        if avi_files:
            avi_path = avi_files[0]


# Inference with clip saving on 0004

In [ ]:
import time
import subprocess
from pathlib import Path
from ultralytics import YOLO

# ---------------- CONFIG ----------------
IDD_ROOT = Path("../../data/idd_temporal_test_1/00004")
MODEL_PATH = "../../runs/three_country_training/road_defect_binary/yolov8s/weights/best.pt"

OUTPUT_ROOT = Path("../../runs/idd_temporal_detected_only/yolov8s")

IMG_SIZE = 640
CONF = 0.50          # 👈 increased confidence
IOU = 0.7
VID_STRIDE = 1
DEVICE = 0           # force GPU
# ---------------------------------------

model = YOLO(MODEL_PATH)
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

print(f"▶ Testing IDD folder: {IDD_ROOT.name}")

for clip_dir in sorted(IDD_ROOT.iterdir()):
    if not clip_dir.is_dir():
        continue

    clip_name = f"00004_{clip_dir.name}"
    print(f"\n▶ Processing clip: {clip_name}")

    detected = False
    frame_count = 0

    # Run inference WITHOUT saving first
    results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=False   # 👈 important
    )

    for r in results:
        frame_count += 1
        if r.boxes is not None and len(r.boxes) > 0:
            detected = True

    if not detected:
        print("   ⏭ No obstacle detected (skipped)")
        continue

    print("   ✅ Obstacle detected → saving video")

    # Re-run inference WITH saving (only for detected clips)
    save_results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=True,
        project=str(OUTPUT_ROOT),
        name=clip_name
    )

    for _ in save_results:
        pass

    # ---------------- Frames → Video (FIXED) ----------------
    run_dir = OUTPUT_ROOT / clip_name
    run_dir.mkdir(parents=True, exist_ok=True)

    mp4_name = "detected.mp4"

    cmd = [
        "ffmpeg", "-y",
        "-framerate", "30",
        "-pattern_type", "glob",
        "-i", "*.jpg",
        "-c:v", "libx264",
        "-pix_fmt", "yuv420p",
        mp4_name
    ]

    subprocess.run(cmd, cwd=run_dir, check=True)

    print(f"   🎥 Saved: {run_dir / mp4_name}")


# Inference with clip saving on 00083

In [ ]:
import time
import subprocess
from pathlib import Path
from ultralytics import YOLO

# ---------------- CONFIG ----------------
IDD_ROOT = Path("../../data/idd_temporal_test_1/00083")
MODEL_PATH = "../../runs/three_country_training/road_defect_binary/yolov8s/weights/best.pt"

OUTPUT_ROOT = Path("../../runs/idd_temporal_detected_only/yolov8s/00083")

IMG_SIZE = 640
CONF = 0.50          # 👈 increased confidence
IOU = 0.7
VID_STRIDE = 1
DEVICE = 0           # force GPU
# ---------------------------------------

model = YOLO(MODEL_PATH)
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

print(f"▶ Testing IDD folder: {IDD_ROOT.name}")

for clip_dir in sorted(IDD_ROOT.iterdir()):
    if not clip_dir.is_dir():
        continue

    clip_name = f"00083_{clip_dir.name}"
    print(f"\n▶ Processing clip: {clip_name}")

    detected = False
    frame_count = 0

    # Run inference WITHOUT saving first
    results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=False   # 👈 important
    )

    for r in results:
        frame_count += 1
        if r.boxes is not None and len(r.boxes) > 0:
            detected = True

    if not detected:
        print("   ⏭ No obstacle detected (skipped)")
        continue

    print("   ✅ Obstacle detected → saving video")

    # Re-run inference WITH saving (only for detected clips)
    save_results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=True,
        project=str(OUTPUT_ROOT),
        name=clip_name
    )

    for _ in save_results:
        pass

    # ---------------- Frames → Video (FIXED) ----------------
    run_dir = OUTPUT_ROOT / clip_name
    run_dir.mkdir(parents=True, exist_ok=True)

    mp4_name = "detected.mp4"

    cmd = [
        "ffmpeg", "-y",
        "-framerate", "30",
        "-pattern_type", "glob",
        "-i", "*.jpg",
        "-c:v", "libx264",
        "-pix_fmt", "yuv420p",
        mp4_name
    ]

    subprocess.run(cmd, cwd=run_dir, check=True)

    print(f"   🎥 Saved: {run_dir / mp4_name}")


# Inferencing on IDD 00083 if obstacle appears in more than 3 frames consecutively and saving timestamps/video clips

In [ ]:
import time
import subprocess
from pathlib import Path
import pandas as pd
from ultralytics import YOLO

# ---------------- CONFIG ----------------
IDD_ROOT = Path("../../data/idd_temporal_test_1/00083")
MODEL_PATH = "../../runs/three_country_training/road_defect_binary/yolov8s/weights/best.pt"

OUTPUT_ROOT = Path("../../runs/idd_temporal_final/yolov8s/00083")
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

IMG_SIZE = 640
CONF = 0.50
IOU = 0.7
VID_STRIDE = 1
DEVICE = 0
FPS_ASSUMED = 30
MIN_FRAMES_FOR_DEFECT = 3
# ---------------------------------------

model = YOLO(MODEL_PATH)

# ---------------- GLOBAL METRICS ----------------
global_frame_idx = 0
total_frames = 0
total_time = 0.0

timestamp_records = []
saved_clip_videos = []

print(f"▶ Processing IDD parent folder: {IDD_ROOT.name}")

# ---------------- PROCESS EACH CLIP ----------------
for clip_dir in sorted(IDD_ROOT.iterdir()):
    if not clip_dir.is_dir():
        continue

    clip_name = f"{IDD_ROOT.name}_{clip_dir.name}"
    print(f"\n▶ Clip: {clip_name}")

    frame_idx_in_clip = 0
    detection_frames = []

    start = time.time()

    results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=False
    )

    for r in results:
        frame_idx_in_clip += 1
        total_frames += 1

        if r.boxes is not None and len(r.boxes) > 0:
            detection_frames.append(global_frame_idx)

        global_frame_idx += 1

    elapsed = time.time() - start
    total_time += elapsed

    # ---------- TEMPORAL FILTER ----------
    if len(detection_frames) < MIN_FRAMES_FOR_DEFECT:
        print("   ⏭ Ignored (fails ≥3-frame rule)")
        continue

    print(f"   ✅ Defect confirmed ({len(detection_frames)} frames)")

    # ---------- LOG TIMESTAMPS ----------
    for gf in detection_frames:
        timestamp_records.append({
            "parent_video": IDD_ROOT.name,
            "clip": clip_dir.name,
            "global_frame": gf,
            "timestamp_sec": round(gf / FPS_ASSUMED, 3)
        })

    # ---------- SAVE CLIP ----------
    save_results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=True,
        project=str(OUTPUT_ROOT),
        name=clip_name
    )

    for _ in save_results:
        pass

    run_dir = OUTPUT_ROOT / clip_name
    mp4_name = "detected.mp4"

    subprocess.run(
        [
            "ffmpeg", "-y",
            "-framerate", str(FPS_ASSUMED),
            "-pattern_type", "glob",
            "-i", "*.jpg",
            "-c:v", "libx264",
            "-pix_fmt", "yuv420p",
            mp4_name
        ],
        cwd=run_dir,
        check=True
    )

    saved_clip_videos.append(run_dir / mp4_name)
    print(f"   🎥 Saved clip video")

# ---------------- MERGE ALL CLIPS ----------------
if saved_clip_videos:
    concat_file = OUTPUT_ROOT / "concat.txt"
    with open(concat_file, "w") as f:
        for v in saved_clip_videos:
            f.write(f"file '{v.resolve()}'\n")

    final_video = OUTPUT_ROOT / "00083_full_detected_video.mp4"

    subprocess.run(
        [
            "ffmpeg", "-y",
            "-f", "concat",
            "-safe", "0",
            "-i", str(concat_file),
            "-c", "copy",
            str(final_video)
        ],
        check=True
    )

    print(f"\n🎬 Merged video saved: {final_video}")

# ---------------- SUMMARY DATAFRAME ----------------
avg_fps = total_frames / total_time if total_time > 0 else 0
video_length_sec = total_frames / FPS_ASSUMED

summary_df = pd.DataFrame([{
    "video_id": IDD_ROOT.name,
    "total_length_sec": round(video_length_sec, 2),
    "processed_frames": total_frames,
    "processed_time_sec": round(total_time, 2),
    "avg_fps": round(avg_fps, 2),
    "num_detected_timestamps": len(timestamp_records)
}])

summary_df.to_csv(OUTPUT_ROOT / "video_summary.csv", index=False)

# ---------------- TIMESTAMP FILE ----------------
timestamps_df = pd.DataFrame(timestamp_records)
timestamps_df.to_csv(OUTPUT_ROOT / "defect_timestamps.csv", index=False)

print("\n📊 Summary:")
print(summary_df)
print(f"\n🕒 Timestamp log saved ({len(timestamp_records)} entries)")


In [9]:
# Loading data saved

defect_data_00083 = pd.read_csv("../../runs/idd_temporal_final/yolov8s/00083/defect_timestamps.csv")
video_summary_00083 = pd.read_csv("../../runs/idd_temporal_final/yolov8s/00083/video_summary.csv")

In [10]:
defect_data_00083.head()

,parent_video,clip,global_frame,timestamp_sec
0,83,161242_leftImg8bit,32,1.067
1,83,161242_leftImg8bit,33,1.100
2,83,161242_leftImg8bit,34,1.133
3,83,161242_leftImg8bit,35,1.167
4,83,161242_leftImg8bit,36,1.200


In [11]:
video_summary_00083.head()

,video_id,total_length_sec,processed_frames,processed_time_sec,avg_fps,num_detected_timestamps
0,83,39.27,1178,15.25,77.27,178


# Inferencing on IDD 00054f obstacle appears in more than 3 frames consecutively and saving timestamps/video clips

In [ ]:
import time
import subprocess
from pathlib import Path
import pandas as pd
from ultralytics import YOLO

# ---------------- CONFIG ----------------
IDD_ROOT = Path("../../data/idd_temporal_test_1/00054")
MODEL_PATH = "../../runs/three_country_training/road_defect_binary/yolov8s/weights/best.pt"

OUTPUT_ROOT = Path("../../runs/idd_temporal_final/yolov8s/00054")
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

IMG_SIZE = 640
CONF = 0.50
IOU = 0.7
VID_STRIDE = 1
DEVICE = 0
FPS_ASSUMED = 30
MIN_FRAMES_FOR_DEFECT = 3
# ---------------------------------------

model = YOLO(MODEL_PATH)

# ---------------- GLOBAL METRICS ----------------
global_frame_idx = 0
total_frames = 0
total_time = 0.0

timestamp_records = []
saved_clip_videos = []

print(f"▶ Processing IDD parent folder: {IDD_ROOT.name}")

# ---------------- PROCESS EACH CLIP ----------------
for clip_dir in sorted(IDD_ROOT.iterdir()):
    if not clip_dir.is_dir():
        continue

    clip_name = f"{IDD_ROOT.name}_{clip_dir.name}"
    print(f"\n▶ Clip: {clip_name}")

    frame_idx_in_clip = 0
    detection_frames = []

    start = time.time()

    results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=False
    )

    for r in results:
        frame_idx_in_clip += 1
        total_frames += 1

        if r.boxes is not None and len(r.boxes) > 0:
            detection_frames.append(global_frame_idx)

        global_frame_idx += 1

    elapsed = time.time() - start
    total_time += elapsed

    # ---------- TEMPORAL FILTER ----------
    if len(detection_frames) < MIN_FRAMES_FOR_DEFECT:
        print("   ⏭ Ignored (fails ≥3-frame rule)")
        continue

    print(f"   ✅ Defect confirmed ({len(detection_frames)} frames)")

    # ---------- LOG TIMESTAMPS ----------
    for gf in detection_frames:
        timestamp_records.append({
            "parent_video": IDD_ROOT.name,
            "clip": clip_dir.name,
            "global_frame": gf,
            "timestamp_sec": round(gf / FPS_ASSUMED, 3)
        })

    # ---------- SAVE CLIP ----------
    save_results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=True,
        project=str(OUTPUT_ROOT),
        name=clip_name
    )

    for _ in save_results:
        pass

    run_dir = OUTPUT_ROOT / clip_name
    mp4_name = "detected.mp4"

    subprocess.run(
        [
            "ffmpeg", "-y",
            "-framerate", str(FPS_ASSUMED),
            "-pattern_type", "glob",
            "-i", "*.jpg",
            "-c:v", "libx264",
            "-pix_fmt", "yuv420p",
            mp4_name
        ],
        cwd=run_dir,
        check=True
    )

    saved_clip_videos.append(run_dir / mp4_name)
    print(f"   🎥 Saved clip video")

# ---------------- MERGE ALL CLIPS ----------------
if saved_clip_videos:
    concat_file = OUTPUT_ROOT / "concat.txt"
    with open(concat_file, "w") as f:
        for v in saved_clip_videos:
            f.write(f"file '{v.resolve()}'\n")

    final_video = OUTPUT_ROOT / "00054_full_detected_video.mp4"

    subprocess.run(
        [
            "ffmpeg", "-y",
            "-f", "concat",
            "-safe", "0",
            "-i", str(concat_file),
            "-c", "copy",
            str(final_video)
        ],
        check=True
    )

    print(f"\n🎬 Merged video saved: {final_video}")

# ---------------- SUMMARY DATAFRAME ----------------
avg_fps = total_frames / total_time if total_time > 0 else 0
video_length_sec = total_frames / FPS_ASSUMED

summary_df = pd.DataFrame([{
    "video_id": IDD_ROOT.name,
    "total_length_sec": round(video_length_sec, 2),
    "processed_frames": total_frames,
    "processed_time_sec": round(total_time, 2),
    "avg_fps": round(avg_fps, 2),
    "num_detected_timestamps": len(timestamp_records)
}])

summary_df.to_csv(OUTPUT_ROOT / "video_summary.csv", index=False)

# ---------------- TIMESTAMP FILE ----------------
timestamps_df = pd.DataFrame(timestamp_records)
timestamps_df.to_csv(OUTPUT_ROOT / "defect_timestamps.csv", index=False)

print("\n📊 Summary:")
print(summary_df)
print(f"\n🕒 Timestamp log saved ({len(timestamp_records)} entries)")


# Inferencing on IDD 00074 obstacle appears in more than 3 frames consecutively and saving timestamps/video clips

In [ ]:
import time
import subprocess
from pathlib import Path
import pandas as pd
from ultralytics import YOLO

# ---------------- CONFIG ----------------
IDD_ROOT = Path("../../data/idd_temporal_test_1/00074")
MODEL_PATH = "../../runs/three_country_training/road_defect_binary/yolov8s/weights/best.pt"

OUTPUT_ROOT = Path("../../runs/idd_temporal_final/yolov8s/00074")
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

IMG_SIZE = 640
CONF = 0.50
IOU = 0.7
VID_STRIDE = 1
DEVICE = 0
FPS_ASSUMED = 30
MIN_FRAMES_FOR_DEFECT = 3
# ---------------------------------------

model = YOLO(MODEL_PATH)

# ---------------- GLOBAL METRICS ----------------
global_frame_idx = 0
total_frames = 0
total_time = 0.0

timestamp_records = []
saved_clip_videos = []

print(f"▶ Processing IDD parent folder: {IDD_ROOT.name}")

# ---------------- PROCESS EACH CLIP ----------------
for clip_dir in sorted(IDD_ROOT.iterdir()):
    if not clip_dir.is_dir():
        continue

    clip_name = f"{IDD_ROOT.name}_{clip_dir.name}"
    print(f"\n▶ Clip: {clip_name}")

    frame_idx_in_clip = 0
    detection_frames = []

    start = time.time()

    results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=False
    )

    for r in results:
        frame_idx_in_clip += 1
        total_frames += 1

        if r.boxes is not None and len(r.boxes) > 0:
            detection_frames.append(global_frame_idx)

        global_frame_idx += 1

    elapsed = time.time() - start
    total_time += elapsed

    # ---------- TEMPORAL FILTER ----------
    if len(detection_frames) < MIN_FRAMES_FOR_DEFECT:
        print("   ⏭ Ignored (fails ≥3-frame rule)")
        continue

    print(f"   ✅ Defect confirmed ({len(detection_frames)} frames)")

    # ---------- LOG TIMESTAMPS ----------
    for gf in detection_frames:
        timestamp_records.append({
            "parent_video": IDD_ROOT.name,
            "clip": clip_dir.name,
            "global_frame": gf,
            "timestamp_sec": round(gf / FPS_ASSUMED, 3)
        })

    # ---------- SAVE CLIP ----------
    save_results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=True,
        project=str(OUTPUT_ROOT),
        name=clip_name
    )

    for _ in save_results:
        pass

    run_dir = OUTPUT_ROOT / clip_name
    mp4_name = "detected.mp4"

    subprocess.run(
        [
            "ffmpeg", "-y",
            "-framerate", str(FPS_ASSUMED),
            "-pattern_type", "glob",
            "-i", "*.jpg",
            "-c:v", "libx264",
            "-pix_fmt", "yuv420p",
            mp4_name
        ],
        cwd=run_dir,
        check=True
    )

    saved_clip_videos.append(run_dir / mp4_name)
    print(f"   🎥 Saved clip video")

# ---------------- MERGE ALL CLIPS ----------------
if saved_clip_videos:
    concat_file = OUTPUT_ROOT / "concat.txt"
    with open(concat_file, "w") as f:
        for v in saved_clip_videos:
            f.write(f"file '{v.resolve()}'\n")

    final_video = OUTPUT_ROOT / "00074_full_detected_video.mp4"

    subprocess.run(
        [
            "ffmpeg", "-y",
            "-f", "concat",
            "-safe", "0",
            "-i", str(concat_file),
            "-c", "copy",
            str(final_video)
        ],
        check=True
    )

    print(f"\n🎬 Merged video saved: {final_video}")

# ---------------- SUMMARY DATAFRAME ----------------
avg_fps = total_frames / total_time if total_time > 0 else 0
video_length_sec = total_frames / FPS_ASSUMED

summary_df = pd.DataFrame([{
    "video_id": IDD_ROOT.name,
    "total_length_sec": round(video_length_sec, 2),
    "processed_frames": total_frames,
    "processed_time_sec": round(total_time, 2),
    "avg_fps": round(avg_fps, 2),
    "num_detected_timestamps": len(timestamp_records)
}])

summary_df.to_csv(OUTPUT_ROOT / "video_summary.csv", index=False)

# ---------------- TIMESTAMP FILE ----------------
timestamps_df = pd.DataFrame(timestamp_records)
timestamps_df.to_csv(OUTPUT_ROOT / "defect_timestamps.csv", index=False)

print("\n📊 Summary:")
print(summary_df)
print(f"\n🕒 Timestamp log saved ({len(timestamp_records)} entries)")


# Inferencing on IDD 00082 obstacle appears in more than 3 frames consecutively and saving timestamps/video clips

In [ ]:
import time
import subprocess
from pathlib import Path
import pandas as pd
from ultralytics import YOLO

# ---------------- CONFIG ----------------
IDD_ROOT = Path("../../data/idd_temporal_test_1/00082")
MODEL_PATH = "../../runs/three_country_training/road_defect_binary/yolov8s/weights/best.pt"

OUTPUT_ROOT = Path("../../runs/idd_temporal_final/yolov8s/00082")
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

IMG_SIZE = 640
CONF = 0.50
IOU = 0.7
VID_STRIDE = 1
DEVICE = 0
FPS_ASSUMED = 30
MIN_FRAMES_FOR_DEFECT = 3
# ---------------------------------------

model = YOLO(MODEL_PATH)

# ---------------- GLOBAL METRICS ----------------
global_frame_idx = 0
total_frames = 0
total_time = 0.0

timestamp_records = []
saved_clip_videos = []

print(f"▶ Processing IDD parent folder: {IDD_ROOT.name}")

# ---------------- PROCESS EACH CLIP ----------------
for clip_dir in sorted(IDD_ROOT.iterdir()):
    if not clip_dir.is_dir():
        continue

    clip_name = f"{IDD_ROOT.name}_{clip_dir.name}"
    print(f"\n▶ Clip: {clip_name}")

    frame_idx_in_clip = 0
    detection_frames = []

    start = time.time()

    results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=False
    )

    for r in results:
        frame_idx_in_clip += 1
        total_frames += 1

        if r.boxes is not None and len(r.boxes) > 0:
            detection_frames.append(global_frame_idx)

        global_frame_idx += 1

    elapsed = time.time() - start
    total_time += elapsed

    # ---------- TEMPORAL FILTER ----------
    if len(detection_frames) < MIN_FRAMES_FOR_DEFECT:
        print("   ⏭ Ignored (fails ≥3-frame rule)")
        continue

    print(f"   ✅ Defect confirmed ({len(detection_frames)} frames)")

    # ---------- LOG TIMESTAMPS ----------
    for gf in detection_frames:
        timestamp_records.append({
            "parent_video": IDD_ROOT.name,
            "clip": clip_dir.name,
            "global_frame": gf,
            "timestamp_sec": round(gf / FPS_ASSUMED, 3)
        })

    # ---------- SAVE CLIP ----------
    save_results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=True,
        project=str(OUTPUT_ROOT),
        name=clip_name
    )

    for _ in save_results:
        pass

    run_dir = OUTPUT_ROOT / clip_name
    mp4_name = "detected.mp4"

    subprocess.run(
        [
            "ffmpeg", "-y",
            "-framerate", str(FPS_ASSUMED),
            "-pattern_type", "glob",
            "-i", "*.jpg",
            "-c:v", "libx264",
            "-pix_fmt", "yuv420p",
            mp4_name
        ],
        cwd=run_dir,
        check=True
    )

    saved_clip_videos.append(run_dir / mp4_name)
    print(f"   🎥 Saved clip video")

# ---------------- MERGE ALL CLIPS ----------------
if saved_clip_videos:
    concat_file = OUTPUT_ROOT / "concat.txt"
    with open(concat_file, "w") as f:
        for v in saved_clip_videos:
            f.write(f"file '{v.resolve()}'\n")

    final_video = OUTPUT_ROOT / "00082_full_detected_video.mp4"

    subprocess.run(
        [
            "ffmpeg", "-y",
            "-f", "concat",
            "-safe", "0",
            "-i", str(concat_file),
            "-c", "copy",
            str(final_video)
        ],
        check=True
    )

    print(f"\n🎬 Merged video saved: {final_video}")

# ---------------- SUMMARY DATAFRAME ----------------
avg_fps = total_frames / total_time if total_time > 0 else 0
video_length_sec = total_frames / FPS_ASSUMED

summary_df = pd.DataFrame([{
    "video_id": IDD_ROOT.name,
    "total_length_sec": round(video_length_sec, 2),
    "processed_frames": total_frames,
    "processed_time_sec": round(total_time, 2),
    "avg_fps": round(avg_fps, 2),
    "num_detected_timestamps": len(timestamp_records)
}])

summary_df.to_csv(OUTPUT_ROOT / "video_summary.csv", index=False)

# ---------------- TIMESTAMP FILE ----------------
timestamps_df = pd.DataFrame(timestamp_records)
timestamps_df.to_csv(OUTPUT_ROOT / "defect_timestamps.csv", index=False)

print("\n📊 Summary:")
print(summary_df)
print(f"\n🕒 Timestamp log saved ({len(timestamp_records)} entries)")


In [7]:
# Load data

# defect_data_00082 = pd.read_csv("../../runs/idd_temporal_final/yolov8s/00082/defect_timestamps.csv")
# video_summary_00082 = pd.read_csv("../../runs/idd_temporal_final/yolov8s/00082/video_summary.csv")

# Inferencing on IDD 00061 obstacle appears in more than 3 frames consecutively and saving timestamps/video clips

In [ ]:
import time
import subprocess
from pathlib import Path
import pandas as pd
from ultralytics import YOLO

# ---------------- CONFIG ----------------
IDD_ROOT = Path("../../data/idd_temporal_test_1/00061")
MODEL_PATH = "../../runs/three_country_training/road_defect_binary/yolov8s/weights/best.pt"

OUTPUT_ROOT = Path("../../runs/idd_temporal_final/yolov8s/00061")
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

IMG_SIZE = 640
CONF = 0.50
IOU = 0.7
VID_STRIDE = 1
DEVICE = 0
FPS_ASSUMED = 30
MIN_FRAMES_FOR_DEFECT = 3
# ---------------------------------------

model = YOLO(MODEL_PATH)

# ---------------- GLOBAL METRICS ----------------
global_frame_idx = 0
total_frames = 0
total_time = 0.0

timestamp_records = []
saved_clip_videos = []

print(f"▶ Processing IDD parent folder: {IDD_ROOT.name}")

# ---------------- PROCESS EACH CLIP ----------------
for clip_dir in sorted(IDD_ROOT.iterdir()):
    if not clip_dir.is_dir():
        continue

    clip_name = f"{IDD_ROOT.name}_{clip_dir.name}"
    print(f"\n▶ Clip: {clip_name}")

    frame_idx_in_clip = 0
    detection_frames = []

    start = time.time()

    results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=False
    )

    for r in results:
        frame_idx_in_clip += 1
        total_frames += 1

        if r.boxes is not None and len(r.boxes) > 0:
            detection_frames.append(global_frame_idx)

        global_frame_idx += 1

    elapsed = time.time() - start
    total_time += elapsed

    # ---------- TEMPORAL FILTER ----------
    if len(detection_frames) < MIN_FRAMES_FOR_DEFECT:
        print("   ⏭ Ignored (fails ≥3-frame rule)")
        continue

    print(f"   ✅ Defect confirmed ({len(detection_frames)} frames)")

    # ---------- LOG TIMESTAMPS ----------
    for gf in detection_frames:
        timestamp_records.append({
            "parent_video": IDD_ROOT.name,
            "clip": clip_dir.name,
            "global_frame": gf,
            "timestamp_sec": round(gf / FPS_ASSUMED, 3)
        })

    # ---------- SAVE CLIP ----------
    save_results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=True,
        project=str(OUTPUT_ROOT),
        name=clip_name
    )

    for _ in save_results:
        pass

    run_dir = OUTPUT_ROOT / clip_name
    mp4_name = "detected.mp4"

    subprocess.run(
        [
            "ffmpeg", "-y",
            "-framerate", str(FPS_ASSUMED),
            "-pattern_type", "glob",
            "-i", "*.jpg",
            "-c:v", "libx264",
            "-pix_fmt", "yuv420p",
            mp4_name
        ],
        cwd=run_dir,
        check=True
    )

    saved_clip_videos.append(run_dir / mp4_name)
    print(f"   🎥 Saved clip video")

# ---------------- MERGE ALL CLIPS ----------------
if saved_clip_videos:
    concat_file = OUTPUT_ROOT / "concat.txt"
    with open(concat_file, "w") as f:
        for v in saved_clip_videos:
            f.write(f"file '{v.resolve()}'\n")

    final_video = OUTPUT_ROOT / "00061_full_detected_video.mp4"

    subprocess.run(
        [
            "ffmpeg", "-y",
            "-f", "concat",
            "-safe", "0",
            "-i", str(concat_file),
            "-c", "copy",
            str(final_video)
        ],
        check=True
    )

    print(f"\n🎬 Merged video saved: {final_video}")

# ---------------- SUMMARY DATAFRAME ----------------
avg_fps = total_frames / total_time if total_time > 0 else 0
video_length_sec = total_frames / FPS_ASSUMED

summary_df = pd.DataFrame([{
    "video_id": IDD_ROOT.name,
    "total_length_sec": round(video_length_sec, 2),
    "processed_frames": total_frames,
    "processed_time_sec": round(total_time, 2),
    "avg_fps": round(avg_fps, 2),
    "num_detected_timestamps": len(timestamp_records)
}])

summary_df.to_csv(OUTPUT_ROOT / "video_summary.csv", index=False)

# ---------------- TIMESTAMP FILE ----------------
timestamps_df = pd.DataFrame(timestamp_records)
timestamps_df.to_csv(OUTPUT_ROOT / "defect_timestamps.csv", index=False)

print("\n📊 Summary:")
print(summary_df)
print(f"\n🕒 Timestamp log saved ({len(timestamp_records)} entries)")


In [8]:
# Loading data saved

defect_data_00061 = pd.read_csv("../../runs/idd_temporal_final/yolov8s/00061/defect_timestamps.csv")
video_summary_00061 = pd.read_csv("../../runs/idd_temporal_final/yolov8s/00061/video_summary.csv")

In [9]:
defect_data_00061.head()

,parent_video,clip,global_frame,timestamp_sec
0,61,000160_leftImg8bit,0,0.000
1,61,000160_leftImg8bit,1,0.033
2,61,000160_leftImg8bit,2,0.067
3,61,000160_leftImg8bit,3,0.100
4,61,000160_leftImg8bit,6,0.200


In [10]:
video_summary_00061.head()

,video_id,total_length_sec,processed_frames,processed_time_sec,avg_fps,num_detected_timestamps
0,61,13.13,394,13.86,28.42,19
